<a href="https://colab.research.google.com/github/mythrikulkarni/Aggie-Assist/blob/main/AggieAssist1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 16

In [ ]:
import json
with open(r"/content/train.json", "r") as read_file:
    train = json.load(read_file)

In [ ]:
train

[{'context': "There are many types of mental health professionals. The variety of providers and their services may be confusing. Each have various levels of education, training, and may have different areas of expertise. Finding the professional who best fits your needs may require some research. To find resources on campus for mental health, here's a helpful resource: https://shcs.ucdavis.edu/services/counseling-services",
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'What is the difference between mental health professionals?',
    'answers': [{'text': "There are many types of mental health professionals. The variety of providers and their services may be confusing. Each have various levels of education, training, and may have different areas of expertise. Finding the professional who best fits your needs may require some research. To find resources on campus for mental health, here's a helpful resource: https://shcs.ucdavis.edu/services/counseling-services",

In [ ]:
with open(r"/content/test.json", "r") as read_file:
    test = json.load(read_file)

In [ ]:
test

[{'context': 'The Student Health and Counseling Services (SHCS) provides a range of mental health services, including individual counseling, group therapy, psychiatric medication management, and crisis support. These services are available to all registered students, and are covered by the student health insurance plan (UC SHIP) and have prices listed on their website for students under another health insurance plan: https://shcs.ucdavis.edu/services/primary-care',
  'qas': [{'id': '00041',
    'is_impossible': False,
    'question': 'What mental health services are provided by the SHCS?',
    'answers': [{'text': 'The Student Health and Counseling Services (SHCS) provides a range of mental health services, including individual counseling, group therapy, psychiatric medication management, and crisis support.',
      'answer_start': 0}]}]},
 {'context': 'Individual Counseling is confidential short-term therapy available to all registered UC Davis students for free at North Hall. UC Davi

In [ ]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [ ]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 40
model_args.evaluate_during_training = True
model_args.n_best_size=1
model_args.num_train_epochs=1

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 1,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3, 
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [ ]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args, use_cuda= False
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [ ]:
!rm -rf outputs


In [ ]:
model.train_model(train, eval_data=test)

add example index and unique id: 100%|██████████| 39/39 [00:00<00:00, 201202.77it/s]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Running Epoch 0 of 1:   0%|          | 0/1 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 41/41 [00:00<00:00, 171.36it/s]

add example index and unique id: 100%|██████████| 41/41 [00:00<00:00, 28039.53it/s]


Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(1,
 {'global_step': [1],
  'correct': [0],
  'similar': [27],
  'incorrect': [13],
  'train_loss': [4.945713043212891],
  'eval_loss': [0.04123212397098541]})

In [ ]:
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
with open(r"/content/predictions.json", "r") as read_file:
    to_predict = json.load(read_file)

In [ ]:
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 18/18 [00:00<00:00, 39527.47it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '00016', 'answer': ['decide if it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which information is worth considering and which', 'up to the readers to think critically about information to decide if it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which information is worth considering and which', 'decide if it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which']}, {'id': '00017', 'answer': ['decision-making by giving you a clear roadma', 'plan can benefit your mental health by reducing anxiety, boosting motivation, and improving decision-making by giving you a clear roadma', 'decision-making by giving you a clear road']}, {'id': '00018', 'answer': ['and family, joining social groups or clubs, volunteering', 'or clubs, volunteering', 'and family, joining social groups']}, {'id': '00019', 'answer': ['stairs instead of the elevator or escalator, if you’re 

In [ ]:
to_predict = [
    {
        "context": "Having little to no energy and Feeling hopeless and helpless, Overusing substances like alcohol or cigarettes, Experiencing hallucinations or uncharacteristic thoughts/behaviors, Thinking about harming or killing yourself or others, Inability to perform daily tasks like studying or biking to school, Experiencing severe mood swings, Having unexplained aches or pains Ares several symptoms that you might be struggling with you mental health. If you would like to recieve help you can call 911 or go to the nearest emergency room. UC Davis SHCS offers crisis counseling services and has an after-hours crisis line at 530-752-2349. You can also contact the National Suicide Prevention Lifeline at 1-800-273-TALK (8255) or use their online chat feature. https://mentalhealth.ucdavis.edu/resources/signs-symptoms-statistics",
        "qas": [
            {
                "question": "What are some indicators that I am struggling with my mental health?",
                "id": "0",
            }
        ],
    }
]

In [ ]:
answers, probabilities = model.predict(to_predict)

print(answers)
     

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 161.44it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['/behaviors, Thinking about harming or killing yourself or others, Inability to perform daily tasks like studying or biking to school, Experiencing severe mood swings, Having unexplained aches or pains Ares several symptoms that you might be struggling with you mental health. If you would like', 'iencing severe mood swings, Having unexplained aches or pains Ares several symptoms that you might be struggling with you mental health. If you would like', '/behaviors, Thinking about harming or killing yourself or others, Inability to perform daily tasks like studying or biking to school, Experiencing severe mood swings, Having unexplained aches or pains Ares several symptoms that you might be struggling with you mental health. If you would like to recieve help you can call 911 or go to']}]


In [ ]:
to_predict = [
  {
    "context": "Clearly a large body of research evidence points to importance of lifestyle factors in medical and psychiatric diseases. Research also suggest that positive modification of lifestyle factors is essential for both improvement and maintaining of physical and mental health. Each individual is unique and hence requires their own unique set of programmes of lifestyle modification. Indeed, it is helpful to recognise that each lifestyle factor is linked to and influences other lifestyle factors through various underlying mechanisms such as improvement in inflammatory processes which can result from inactivity, overconsumption of unhealthy/inflammatory foods, obesity and smoking. Whilst it may be easier to take a step by step approach by modifying of one lifestyle factor at one time, efforts should be made to modify all lifestyle factors in conjunctions, since they tend to work synergistically and are likely to have greater effect on positively modify morbidity and mortality related to physical and mental health.  Source: https://www.psychiatria-danubina.com/UserDocsImages/pdf/dnb_vol31_noSuppl%203/dnb_vol31_noSuppl%203_217.pdf",
    "qas": [
      {
        "id": "00033",
        "question": "How can obesity impact mental health?",
      }
    ]
  }
]

In [ ]:
answers, probabilities = model.predict(to_predict)

print(answers)
     

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1997.29it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '00033', 'answer': ['suggest that positive modification of lifestyle factors is essential for both improvement and maintaining of physical and mental health. Each individual is unique and hence requires their own', 'lifestyle factors in medical and psychiatric diseases. Research also suggest that positive modification of lifestyle factors is essential for both improvement and maintaining of physical and mental health. Each individual is unique and hence requires their own', 'suggest that positive modification of lifestyle factors is essential for both improvement and maintaining of physical and mental health. Each individual is unique and hence requires their own unique set of programmes of lifestyle modification. Indeed, it is helpful to recognise that each lifestyle factor is linked to and influences']}]


In [ ]:
to_predict = [
  {
    "context": "Anyone can post anything online. It's up to the readers to think critically about information to decide if it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which information is worth considering and which information doesn't pass the test.\nIt's so easy to spread information online, and we can't assume that the source we're reading or watching is the original source. Before you can evaluate anything, you need to know where the claims come from and determine if it's a reliable source.\nReliable sources on the coronavirus and COVID-19 illness include the World Health Organization (WHO), different US governement agencies including the National Institutes of Health (NIH) and the Centers for Disease Control and Prevention(CDC), and well-known universities (including any in the UC system). Scientists and researchers at universities and other institutions and health sciences professionals are also important sources of information. \nHeadlines and titles need to be simple and short, so they can sometimes be misleading on their own. They may also be purposefully shocking or controversial to entice you to click to read more. \nWebsites that belong to governments, government-funded agencies, well-known health providers, universities, or groups of medical professionals are generally the most reliable. Be careful of sites that don't list the author or don't tell you much about themselves. Double-check the author and the credentials of any 'experts' quoted or cited to see if they are actually experts in the topic being discussed. \nCan you find other evidence from other sources to support the claims? Does the author list their sources? Are the sources legitimate, like research journals? \nUnhelpful sources want you to feel a certain way: scared, angry, distrustful, vulnerable. Phrases like \"Skyrocketing rates of infection\" are sensational and they're meant to make people feel afraid. On the other hand, factual statements like \"We have 50 new cases in BC\" give you important information without playing into your emotions. Helpful news or information sources want you feel informed and empowered. Who is paying for the site? What products or services are being advertised? What links or other sites do they promote? Are you learning something from a story or experience, or are you simply being directed to buy a product or service? Social media can be a great way to keep in touch with family and friends, especially as we practice physical distancing or self-isolation. Social media can also be an important way to learn from health experts and connect to support services and mutual aid groups in your community. Like any other source of information, it's important to use good critical thinking skills when you use social media. Studies find that searching for health information on social media generally has very mixed results. You'll see both very poor-quality sources next to good-quality sources, and it's up to you to decide which is which. In addition, social media posts tend to be short and rarely capture everything you need to consider. They are generally useful as a way to start learning or exploring different points of view rather than a place to gather all the information. Visit www.mediasmarts.ca to learn more about digital and media literacy. Visit www.healthnewsreview.org to learn more about thinking critically around health claims (this resource is no longer updated daily, but all toolkits and lessons are still available). This page is adapted from the Evaluating Mental Health and Substance Use Information info sheet.",
    "qas": [
      {
        "id": "00016",
        "question": "How can I tell if health information is good or bad?",
      }
    ]
  }
]

In [ ]:
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2534.32it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '00016', 'answer': ['decide if it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which information is worth considering and which', 'up to the readers to think critically about information to decide if it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which information is worth considering and which', 'decide if it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which']}]


In [ ]:
to_predict = [
  {
    "context": "The Student Health and Counseling Services (SHCS) provides a range of mental health services, including individual counseling, group therapy, psychiatric medication management, and crisis support. These services are available to all registered students, and are covered by the student health insurance plan (UC SHIP) and have prices listed on their website for students under another health insurance plan: https://shcs.ucdavis.edu/services/primary-care",
    "qas": [
      {
        "id": "00041",
        "question": "What mental health services are provided by the SHCS?",
      }
    ]
  }
]

In [ ]:
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2295.73it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '00041', 'answer': ['cs.ucdavis.', 'SHCS) provides a range of mental health services, including individual counseling, group therapy, psychiatric medication management, and crisis support. These services are available to all registered students, and are covered by the student health insurance plan (UC SHIP) and have prices listed on their website for students under another health insurance plan: https:/', 'SHCS) provides a range of mental health services, including individual counseling, group therapy, psychiatric medication management, and crisis support. These services are available to all registered students, and are covered by the student health insurance plan (UC SHIP) and have prices listed on their website for students under another health insurance plan: https://']}]


In [ ]:
to_predict = [
  {
    "context": "If you or someone you know is in immediate danger, call 911 or go to the nearest emergency room. UC Davis SHCS offers crisis counseling services and has an after-hours crisis line at 530-752-2349. You can also contact the National Suicide Prevention Lifeline at 1-800-273-TALK (8255) or use their online chat feature.",
    "qas": [
      {
        "id": "000078",
        "question": "I'm experiencing suicidal thoughts or know someone who is. What should I do and where can I find immediate help?",
      }
    ]
  }
]

In [ ]:
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 763.71it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '000078', 'answer': ['counseling services and has an after-hours crisis line at 530-752-2349. You can', 'counseling services and has an after-hours crisis line at 530-752-2349']}]


In [ ]:
pip install bert-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install load
!pip install evaluate
from evaluate import load
bertscore = load("bertscore")
predictions = ["iencing hallucinations or uncharacteristic thoughts/behaviors, Thinking about harming or killing yourself or others, Inability to perform daily tasks like studying or biking to school, Experiencing severe mood swings, Having unexplained aches or pains Ares several symptoms that you might be struggling with you mental health. If you would like', 'iencing', 'iencing severe mood swings, Having unexplained aches or pains Ares several symptoms that you might be struggling with you mental health. If you would like", "suggest that positive modification of lifestyle factors is essential for both improvement and maintaining of physical and mental health. Each individual is unique and hence requires their own', 'lifestyle factors in medical and psychiatric diseases. Research also suggest that positive modification of lifestyle factors is essential for both improvement and maintaining of physical and mental health. Each individual is unique and hence requires their own', 'suggest that positive modification of lifestyle factors is essential for both improvement and maintaining of physical and mental health. Each individual is unique and hence requires their own unique set of programmes of lifestyle modification. Indeed, it is helpful to recognise that each lifestyle factor is linked to and influences", "decide if it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which information is worth considering and which', 'up to the readers to think critically about information to decide if it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which information is worth considering and which', 'decide if it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which", "'cs.ucdavis.', 'SHCS) provides a range of mental health services, including individual counseling, group therapy, psychiatric medication management, and crisis support. These services are available to all registered students, and are covered by the student health insurance plan (UC SHIP) and have prices listed on their website for students under another health insurance plan: https:/', 'SHCS) provides a range of mental health services, including individual counseling, group therapy, psychiatric medication management, and crisis support. These services are available to all registered students, and are covered by the student health insurance plan (UC SHIP) and have prices listed on their website for students under another health insurance plan: https://'", "counseling services and has an after-hours crisis line at 530-752-2349. You can', 'counseling services and has an after-hours crisis line at 530-752-2349"]
references = ["Having little to no energy and Feeling hopeless and helpless, Overusing substances like alcohol or cigarettes, Experiencing hallucinations or uncharacteristic thoughts/behaviors, Thinking about harming or killing yourself or others, Inability to perform daily tasks like studying or biking to school, Experiencing severe mood swings, Having unexplained aches or pains Ares several symptoms that you might be struggling with you mental health. If you would like to recieve help you can call 911 or go to the nearest emergency room. UC Davis SHCS offers crisis counseling services and has an after-hours crisis line at 530-752-2349. You can also contact the National Suicide Prevention Lifeline at 1-800-273-TALK (8255) or use their online chat feature. https://mentalhealth.ucdavis.edu/resources/signs-symptoms-statistics", "Clearly a large body of research evidence points to importance of lifestyle factors in medical and psychiatric diseases. Research also suggest that positive modification of lifestyle factors is essential for both improvement and maintaining of physical and mental health.", "Anyone can post anything online. It's up to the readers to think critically about information to decide if it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which information is worth considering and which information doesn't pass the test.\nIt's so easy to spread information online, and we can't assume that the source we're reading or watching is the original source. Before you can evaluate anything, you need to know where the claims come from and determine if it's a reliable source.\nReliable sources on the coronavirus and COVID-19 illness include the World Health Organization (WHO), different US governement agencies including the National Institutes of Health (NIH) and the Centers for Disease Control and Prevention(CDC), and well-known universities (including any in the UC system). Scientists and researchers at universities and other institutions and health sciences professionals are also important sources of information. \nHeadlines and titles need to be simple and short, so they can sometimes be misleading on their own. They may also be purposefully shocking or controversial to entice you to click to read more. \nWebsites that belong to governments, government-funded agencies, well-known health providers, universities, or groups of medical professionals are generally the most reliable. Be careful of sites that don't list the author or don't tell you much about themselves. Double-check the author and the credentials of any 'experts' quoted or cited to see if they are actually experts in the topic being discussed. \nCan you find other evidence from other sources to support the claims? Does the author list their sources? Are the sources legitimate, like research journals? \nUnhelpful sources want you to feel a certain way: scared, angry, distrustful, vulnerable. Phrases like \"Skyrocketing rates of infection\" are sensational and they're meant to make people feel afraid. On the other hand, factual statements like \"We have 50 new cases in BC\" give you important information without playing into your emotions. Helpful news or information sources want you feel informed and empowered. Who is paying for the site? What products or services are being advertised? What links or other sites do they promote? Are you learning something from a story or experience, or are you simply being directed to buy a product or service? Social media can be a great way to keep in touch with family and friends, especially as we practice physical distancing or self-isolation. Social media can also be an important way to learn from health experts and connect to support services and mutual aid groups in your community. Like any other source of information, it's important to use good critical thinking skills when you use social media. Studies find that searching for health information on social media generally has very mixed results. You'll see both very poor-quality sources next to good-quality sources, and it's up to you to decide which is which. In addition, social media posts tend to be short and rarely capture everything you need to consider. They are generally useful as a way to start learning or exploring different points of view rather than a place to gather all the information. Visit www.mediasmarts.ca to learn more about digital and media literacy. Visit www.healthnewsreview.org to learn more about thinking critically around health claims (this resource is no longer updated daily, but all toolkits and lessons are still available). This page is adapted from the Evaluating Mental Health and Substance Use Information info sheet.", "The Student Health and Counseling Services (SHCS) provides a range of mental health services, including individual counseling, group therapy, psychiatric medication management, and crisis support. These services are available to all registered students, and are covered by the student health insurance plan (UC SHIP) and have prices listed on their website for students under another health insurance plan: https://shcs.ucdavis.edu/services/primary-care", "If you or someone you know is in immediate danger, call 911 or go to the nearest emergency room. UC Davis SHCS offers crisis counseling services and has an after-hours crisis line at 530-752-2349. You can also contact the National Suicide Prevention Lifeline at 1-800-273-TALK (8255) or use their online chat feature."]
results = bertscore.compute(predictions=predictions, references=references, lang="en")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for load: filename=load-2020.12.3-py3-none-any.whl size=1296 sha256=3e487ca09edaf65717acf8ca13aa58b4e2082173014d078aa7c1c3cb2474a3ee
  Stored in directory: /root/.cache/pip/wheels/10/ac/75/b2ed5040bdd186cacbb1b3dd1bb5070c76b538f83c9ec2b335
Successfully built load
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.1 MB/s eta 0:00:00


In [ ]:
print(results)

{'precision': [0.9190394878387451, 0.8666793704032898, 0.8711733818054199, 0.9140093326568604, 0.874667763710022], 'recall': [0.8368430733680725, 0.9052644968032837, 0.7864521741867065, 0.9174199104309082, 0.8352051377296448], 'f1': [0.8760174512863159, 0.8855518698692322, 0.8266476988792419, 0.9157114624977112, 0.8544809818267822], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.29.2)'}
